In [1]:
from experiment_project.utils.initial.util import init_sys_env
from experiment_project.utils.files.read import read_yaml
import dspy
init_sys_env()
secret_env_file = '/mnt/d/project/dy/extra/autogen/env_secret_config.yaml'

api_configs = read_yaml(secret_env_file)

model_config = api_configs.get('openai')
# turbo = dspy.OpenAI(model=model_config.get('model'), max_tokens=2048,api_key=model_config.get('api_key'))
turbo = dspy.OllamaLocal(model='qwen:32b-text-v1.5-q4_0',base_url='http://192.168.0.75:11434')
dspy.settings.configure(lm=turbo)

In [2]:
turbo('Hello? 你好')

['你好！有什么我可以帮忙的吗？']

In [2]:
# 最简单的提问回答案例 
import dspy

# 定义 Signature
class Entities(dspy.Signature):
    question = dspy.InputField()
    entities = dspy.OutputField(desc="获取实体和实体的类型" )
    condition = dspy.OutputField(desc="识别并列出所有重要的实体。实体可以是人名、地名、组织名、日期、事件等. 对于每个实体，请指定其类型（例如人名、地点、组织等）. 不能有重复的相同的内容" )
    return_example = dspy.OutputField(desc="返回的示例: [{'entity_name':'','entity_rtype':''}]")
    
class BuildKG(dspy.Signature):
    question = dspy.InputField()
    entities = dspy.InputField()
    answer = dspy.OutputField(desc="请你根据我传递过来的每一个entity_name和entity_type,根据构建知识图谱的原理,构建出一个完整的数据体系. 如果某个实体没有在文本中明确定义，你应该根据上下文推断其可能的属性和与其他实体的关联关系，并尽可能提供详细信息。", )
    return_type = dspy.OutputField(desc="""返回的结果变成一个json结果,返回案例是: [
        {
            "entity_name": "",
            "entity_type": "",
            "description": "",
            "relationships": [
                {"related_entity": "", "relation_type": ""},
                {"related_entity": "", "relation_type": ""}
            ]
        }""" )
# 创建任务模块
class KGModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.entities = dspy.Predict(Entities)
        self.kg = dspy.Predict(BuildKG)

    def forward(self, question:str):
        entities = self.entities(question=question).entities
        print(entities)
        answer = self.kg(question=question,entities=entities)
        return answer


# 创建 QAModule 实例
kg_module = KGModule()
# 定义问题
question = """
我与父亲不相见已二年余了，我最不能忘记的是他的背影。那年冬天，祖母死了，父亲的差使也交卸了，正是祸不单行的日子，我从北京到徐州，打算跟着父亲奔丧回家。到徐州见着父亲，看见满院狼藉的东西，又想起祖母，不禁簌簌地流下眼泪。父亲说，“事已如此，不必难过，好在天无绝人之路！”
　　回家变卖典质，父亲还了亏空；又借钱办了丧事。这些日子，家中光景很是惨淡，一半为了丧事，一半为了父亲赋闲。丧事完毕，父亲要到南京谋事，我也要回北京念书，我们便同行。
　　到南京时，有朋友约去游逛，勾留了一日；第二日上午便须渡江到浦口，下午上车北去。父亲因为事忙，本已说定不送我，叫旅馆里一个熟识的茶房陪我同去。他再三嘱咐茶房，甚是仔细。但他终于不放心，怕茶房不妥帖；颇踌躇了一会。其实我那年已二十岁，北京已来往过两三次，是没有甚么要紧的了。他踌躇了一会，终于决定还是自己送我去。我两三回劝他不必去；他只说，“不要紧，他们去不好！”
　　我们过了江，进了车站。我买票，他忙着照看行李。行李太多了，得向脚夫行些小费，才可过去。他便又忙着和他们讲价钱。我那时真是聪明过分，总觉他说话不大漂亮，非自己插嘴不可。但他终于讲定了价钱；就送我上车。他给我拣定了靠车门的一张椅子；我将他给我做的紫毛大衣铺好坐位。他嘱我路上小心，夜里警醒些，不要受凉。又嘱托茶房好好照应我。我心里暗笑他的迂；他们只认得钱，托他们直是白托！而且我这样大年纪的人，难道还不能料理自己么？唉，我现在想想，那时真是太聪明了！
　　我说道，“爸爸，你走吧。”他望车外看了看，说，“我买几个橘子去。你就在此地，不要走动。”我看那边月台的栅栏外有几个卖东西的等着顾客。走到那边月台，须穿过铁道，须跳下去又爬上去。父亲是一个胖子，走过去自然要费事些。我本来要去的，他不肯，只好让他去。我看见他戴着黑布小帽，穿着黑布大马褂，深青布棉袍，蹒跚地走到铁道边，慢慢探身下去，尚不大难。可是他穿过铁道，要爬上那边月台，就不容易了。他用两手攀着上面，两脚再向上缩；他肥胖的身子向左微倾，显出努力的样子。这时我看见他的背影，我的泪很快地流下来了。我赶紧拭干了泪，怕他看见，也怕别人看见。我再向外看时，他已抱了朱红的橘子望回走了。过铁道时，他先将橘子散放在地上，自己慢慢爬下，再抱起橘子走。到这边时，我赶紧去搀他。他和我走到车上，将橘子一股脑儿放在我的皮大衣上。于是扑扑衣上的泥土，心里很轻松似的，过一会说，“我走了；到那边来信！”我望着他走出去。他走了几步，回过头看见我，说，“进去吧，里边没人。”等他的背影混入来来往往的人里，再找不着了，我便进来坐下，我的眼泪又来了。
　　近几年来，父亲和我都是东奔西走，家中光景是一日不如一日。他少年出外谋生，独力支持，做了许多大事。那知老境却如此颓唐！他触目伤怀，自然情不能自已。情郁于中，自然要发之于外；家庭琐屑便往往触他之怒。他待我渐渐不同往日。但最近两年的不见，他终于忘却我的不好，只是惦记着我，惦记着我的儿子。我北来后，他写了一信给我，信中说道，“我身体平安，惟膀子疼痛利害，举箸提笔，诸多不便，大约大去之期不远矣。”我读到此处，在晶莹的泪光中，又看见那肥胖的，青布棉袍，黑布马褂的背影。唉！我不知何时再能与他相见！
　
　　1925年10月在北京。
"""
# 执行任务并获取答案
response = kg_module.forward(question)
print(response.answer)  

父亲, 我, 茶房, 橘子, 朱红, 皮大衣, 铁道, 信, 大去之期, 东奔西走, 他, 这样, 他们, 月台, 那边, 父亲和我, 我们
Relations: 父亲-送, 橘子-买, 茶房-照应, 朱红-橘子, 皮大衣-放, 铁道-穿过, 信-写, 大去之期-不远, 东奔西走-父亲和我, 我们-东
父亲


In [3]:
turbo.inspect_history(n=3)




Given the fields `question`, produce the fields `entities`, `condition`, `return_example`.

---

Follow the following format.

Question: ${question}

Entities: 获取实体和实体的类型

Condition: 识别并列出所有重要的实体。实体可以是人名、地名、组织名、日期、事件等. 对于每个实体，请指定其类型（例如人名、地点、组织等）. 不能有重复的相同的内容

Return Example: 返回的示例: [{'entity_name':'','entity_rtype':''}]

---

Question: 我与父亲不相见已二年余了，我最不能忘记的是他的背影。那年冬天，祖母死了，父亲的差使也交卸了，正是祸不单行的日子，我从北京到徐州，打算跟着父亲奔丧回家。到徐州见着父亲，看见满院狼藉的东西，又想起祖母，不禁簌簌地流下眼泪。父亲说，“事已如此，不必难过，好在天无绝人之路！” 回家变卖典质，父亲还了亏空；又借钱办了丧事。这些日子，家中光景很是惨淡，一半为了丧事，一半为了父亲赋闲。丧事完毕，父亲要到南京谋事，我也要回北京念书，我们便同行。 到南京时，有朋友约去游逛，勾留了一日；第二日上午便须渡江到浦口，下午上车北去。父亲因为事忙，本已说定不送我，叫旅馆里一个熟识的茶房陪我同去。他再三嘱咐茶房，甚是仔细。但他终于不放心，怕茶房不妥帖；颇踌躇了一会。其实我那年已二十岁，北京已来往过两三次，是没有甚么要紧的了。他踌躇了一会，终于决定还是自己送我去。我两三回劝他不必去；他只说，“不要紧，他们去不好！” 我们过了江，进了车站。我买票，他忙着照看行李。行李太多了，得向脚夫行些小费，才可过去。他便又忙着和他们讲价钱。我那时真是聪明过分，总觉他说话不大漂亮，非自己插嘴不可。但他终于讲定了价钱；就送我上车。他给我拣定了靠车门的一张椅子；我将他给我做的紫毛大衣铺好坐位。他嘱我路上小心，夜里警醒些，不要受凉。又嘱托茶房好好照应我。我心里暗笑他的迂；他们只认得钱，托他们直是白托！而且我这样大年纪的人，难道还不能料理自己么？唉，我现在想想，那时真是太聪明了！ 我说道，“爸爸，你走吧。”他望车外看了看，说，“我买几个橘子去。你就在此地

'\n\n\nGiven the fields `question`, produce the fields `entities`, `condition`, `return_example`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\n\nEntities: 获取实体和实体的类型\n\nCondition: 识别并列出所有重要的实体。实体可以是人名、地名、组织名、日期、事件等. 对于每个实体，请指定其类型（例如人名、地点、组织等）. 不能有重复的相同的内容\n\nReturn Example: 返回的示例: [{\'entity_name\':\'\',\'entity_rtype\':\'\'}]\n\n---\n\nQuestion: 我与父亲不相见已二年余了，我最不能忘记的是他的背影。那年冬天，祖母死了，父亲的差使也交卸了，正是祸不单行的日子，我从北京到徐州，打算跟着父亲奔丧回家。到徐州见着父亲，看见满院狼藉的东西，又想起祖母，不禁簌簌地流下眼泪。父亲说，“事已如此，不必难过，好在天无绝人之路！” 回家变卖典质，父亲还了亏空；又借钱办了丧事。这些日子，家中光景很是惨淡，一半为了丧事，一半为了父亲赋闲。丧事完毕，父亲要到南京谋事，我也要回北京念书，我们便同行。 到南京时，有朋友约去游逛，勾留了一日；第二日上午便须渡江到浦口，下午上车北去。父亲因为事忙，本已说定不送我，叫旅馆里一个熟识的茶房陪我同去。他再三嘱咐茶房，甚是仔细。但他终于不放心，怕茶房不妥帖；颇踌躇了一会。其实我那年已二十岁，北京已来往过两三次，是没有甚么要紧的了。他踌躇了一会，终于决定还是自己送我去。我两三回劝他不必去；他只说，“不要紧，他们去不好！” 我们过了江，进了车站。我买票，他忙着照看行李。行李太多了，得向脚夫行些小费，才可过去。他便又忙着和他们讲价钱。我那时真是聪明过分，总觉他说话不大漂亮，非自己插嘴不可。但他终于讲定了价钱；就送我上车。他给我拣定了靠车门的一张椅子；我将他给我做的紫毛大衣铺好坐位。他嘱我路上小心，夜里警醒些，不要受凉。又嘱托茶房好好照应我。我心里暗笑他的迂；他们只认得钱，托他们直是白托！而且我这样大年纪的人，难道还不能料理自己么？唉，我现在想想，那时真是太聪明了！ 我说道，“爸爸，你